In [57]:
# Library Imports

from pyspark.mllib.classification import LogisticRegressionWithSGD

from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark import SparkContext

from pyspark.mllib.feature import StandardScaler

from numpy import array

# Load and parse the data

def parsePoint(line):

    values = [float(x) for x in line.split(',')]

    return LabeledPoint(values[11], values[0:10])

sc = SparkContext("local", "LogisticRegressionWithSGD")

sparkHome = os.environ.get('SPARK_HOME')
fileLocation = sparkHome + "/Assignment1/data.csv";

data = sc.textFile(fileLocation)

parsedData = data.map(parsePoint)

#Split training/test sets

(trainingData, testData) = parsedData.randomSplit([0.6, 0.4])

#Scaling features 

label = trainingData.map(lambda x: x.label)

features = trainingData.map(lambda x: x.features)

scaler1 = StandardScaler().fit(features)

trainingData = label.zip(scaler1.transform(features))

trainingData = trainingData.map(lambda (x,y) : LabeledPoint(x,y))

# Build the model with the parsedData for 100 iterations using regularization 

#parameter as 0.01 and regularization type as ‘l1’

model = LogisticRegressionWithSGD.train(trainingData, 

iterations=100,regParam=0.01,regType='l1')

# Evaluate the model on training data

labelsAndPreds = testData.map(lambda p: (p.label, model.predict(p.features)))

#Calculating the Training error using the following formula

trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(testData.count())

#Calculating the Accuracy using the following formula

accuracy = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(testData.count())

#Print the Training Error and Accuracy calculated

print("Training Error = " + str(trainErr))

print("Accuracy = " + str(accuracy))

Training Error = 0.0343607882769
Accuracy = 0.965639211723


In [56]:
sc.stop()